__Imports__

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import math
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.regularizers import l1

__Read in Data__

In [3]:
train = pd.read_csv("C:/Users/Michael/Github/transactions_train.csv")

__All the transactions made by one customer__

In [4]:
# c_id = train.customer_id[0]
# df = pd.DataFrame(train[train.customer_id==c_id])
df = train.sample(1000)
#we already know the customer's identification
df.drop(['price','sales_channel_id'],axis=1,inplace=True)
df.head()

,t_dat,customer_id,article_id
30387892,2020-08-15,a36762834e471ca01c73ed476a77c538b624e0238ec456...,891050003
17234870,2019-09-28,ce8685caace2236417a475bbb4374fbf38c79fedb14421...,746775001
16386392,2019-09-08,55bddffcd7bdbea6ac65042ab1268d86ba6b955c10d8eb...,767925004
17391051,2019-10-02,15c36fc14180eb35d138776ef6561e25ab24d58800fcfc...,626054001
5932115,2019-02-07,82b31f679f436c0e54940334125c3b1691388b7ad60355...,507910027


__Make the index the Date__

In [5]:
df['t_dat'] = pd.to_datetime(df['t_dat'].apply(lambda x:x.split()[0]))
df.set_index(['t_dat','customer_id'],drop=True,inplace=True)
df.head()

,,article_id
t_dat,customer_id,
2020-08-15,a36762834e471ca01c73ed476a77c538b624e0238ec456f99674e13cfe65e9ed,891050003
2019-09-28,ce8685caace2236417a475bbb4374fbf38c79fedb1442186d76ca892ed4a9743,746775001
2019-09-08,55bddffcd7bdbea6ac65042ab1268d86ba6b955c10d8eb28d57b62dc57f62630,767925004
2019-10-02,15c36fc14180eb35d138776ef6561e25ab24d58800fcfc6c6558c4f503c0bf24,626054001
2019-02-07,82b31f679f436c0e54940334125c3b1691388b7ad603552e0c11e2d2b241dddd,507910027


In [6]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['article_id'].astype(str))
#tokenizer all articles and dictonary them
lst = tokenizer.sequences_to_texts([tokenizer.index_word.keys()])[0].split()
lst
# top12 = '0' + ' 0'.join(lst)
# top12
top_items = [int(i) for i in lst]

In [7]:
df = pd.get_dummies(df.article_id).groupby(['t_dat','customer_id']).sum()
df[top_items]

562245001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               706016001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               706016003  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               799365011  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               746775001  \
t_dat      customer_id         

In [8]:
# df = df[top_items]

__Adjusting the number of steps__

In [9]:
def create_sequence(dataset):
    sequences = []
    labels = []
    start_idx = 0
    for stop_idx in range(3,len(dataset)): 
        sequences.append(dataset.iloc[start_idx:stop_idx])
        labels.append(dataset.iloc[stop_idx])
        start_idx += 1
    return (np.array(sequences),np.array(labels))

__Creating Sequences for Client__

In [10]:
train_seq,train_label = create_sequence(df)
# test_seq, test_label = create_sequence(test_volume)

In [11]:
train_seq.shape, train_label.shape

((997, 3, 969), (997, 969))

In [12]:
model = Sequential()
model.add(LSTM(units=64,input_shape=(train_seq.shape[1],train_seq.shape[2]),return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=32,return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=24))
model.add(Dense(len(top_items),activation='softmax'))

In [13]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['mse'])

In [14]:
model.fit(train_seq, train_label,epochs=50)

Epoch 1/50
32/32 [==============================] - 4s 6ms/step - loss: 0.0010 - mse: 0.0010
Epoch 2/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 3/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 4/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 5/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 6/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 7/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 8/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 9/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 10/50
32/32 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010
Epoch 11/50
32/32 [==============================] - 0s 5ms/step - lo

In [15]:
df_slic_data= pd.DataFrame(df)

In [16]:
df_slic_data

111593001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               111609001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               158340001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               188183020  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               189616001  \
t_dat      customer_id         

In [17]:
dates = pd.date_range(df_slic_data.index.get_level_values('t_dat')[-1],periods=8).tolist()

In [18]:
customers = np.unique(df.index.get_level_values('customer_id'))

In [19]:
date_arr = []
customer_arr = []
for date in dates:
    for cust in customers:
        date_arr.append(date)
        customer_arr.append(cust)

In [20]:
df_test = pd.DataFrame(index = [date_arr,customer_arr],columns=df_slic_data.columns).fillna(0)

__Next 7 days__

In [21]:
# pd.date_range(df_slic_data.index.get_level_values('t_dat')[-1], periods=8).to_list()

In [22]:
# df_slic_data = df_slic_data.append(pd.DataFrame(columns=top_items,
#                                                 index=pd.date_range(start=df_slic_data.index.get_level_values('t_dat')[-1], 
#                                                                     periods=8, freq='D', closed='right')))

In [23]:
df_slic_data.fillna(0,inplace=True)

In [24]:
df_slic_data

111593001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               111609001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               158340001  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               188183020  \
t_dat      customer_id                                                     
2018-09-20 41677ab90f06dbcf59ac17c4493b713f924eeaf8abe02f6...          0   
2018-09-21 bdb026a57b8f4c6c38ae45d7a085c22c4e215b578455b25...          0   
2018-09-22 be2e175f9e27f91804cff00b3f25ae40a7494874dc32ce6...          0   
2018-09-23 2312a1ccfea30b1c4257f87bba32713f388b451b18b5194...          0   
           a63566f24ec94ffa7ed75cd42917942b02145b61887ef46...          0   
...                                                                  ...   
2020-09-18 b617ca7c3ed1b5a211f68b7341712052ef8c0d45ce33a35...          0   
           c9b5bbcbfe543987d31f1a2bc55335f1cf7b424236be16d...          0   
2020-09-19 4a5d33814e5356fabda22de5c1dc9891ecb4af25daadb54...          0   
2020-09-20 7ac274fc84133109b1617ba399baecb6c029b16f2f7e96e...          0   
           954c899a84b47328b7c1bf892f14a1da75eb726432c5140...          0   

                                                               189616001  \
t_dat      customer_id         

In [25]:
# upcoming_prediction = pd.DataFrame(df_slic_data.iloc[7:])
# upcoming_prediction.index=pd.to_datetime(upcoming_prediction.index)

In [26]:
# upcoming_prediction

In [27]:
test_seq,test_label = create_sequence(df_test)

In [28]:
test_seq.shape

(7989, 3, 969)

In [29]:
# model.predict(np.asarray(test_seq).astype(np.float32))
prediction = model.predict(test_seq)

In [30]:
prediction

array([[0.00103094, 0.00103164, 0.00103104, ..., 0.00103306, 0.00103055,
        0.00103029],
       [0.00103094, 0.00103164, 0.00103104, ..., 0.00103306, 0.00103055,
        0.00103029],
       [0.00103094, 0.00103164, 0.00103104, ..., 0.00103306, 0.00103055,
        0.00103029],
       ...,
       [0.00103094, 0.00103164, 0.00103104, ..., 0.00103306, 0.00103055,
        0.00103029],
       [0.00103094, 0.00103164, 0.00103104, ..., 0.00103306, 0.00103055,
        0.00103029],
       [0.00103094, 0.00103164, 0.00103104, ..., 0.00103306, 0.00103055,
        0.00103029]], dtype=float32)

__Top 12 items by client__

In [31]:
preds = [np.argsort(i)[::-1][:12] for i in prediction]

In [36]:
preds[0].astype(str)

array(['735', '120', '416', '418', '319', '805', '272', '635', '696',
       '579', '830', '25'], dtype='<U21')

In [40]:
tokenizer.sequences_to_texts([[735,120,416,418,319,805,272,635,696,579,830,25]])

['759391001 585123002 835607002 821140001 806528003 767032001 941005002 784650008 693242031 764073004 919438002 723469001']

In [218]:
df.tail()

,,153115019,156231001,158340001,186262001,212629004,226959009,233091021,240561001,243613004,282832001,...,902225006,903306002,905122001,905803002,906645010,914441005,917606004,918836001,934536001,938624001
t_dat,customer_id,,,,,,,,,,,,,,,,,,,,,
2020-09-13,b62103ad7f1c72335bc5a30dd2d0ae17600ee7f444d2fea2e85e116ed5cd6454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-09-15,a58dfbbc04bf407b8403dc809724e0951ba6948466a613ffd1281e0b6b3ddc53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-09-18,b8d440591d1e38d309e5a1b5a368c26e4c0dc68d83d773ce974d69f20b76389f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-09-19,5a6b49bb2ff6bcf9dc5cadbe72515d2dfdb0ffba01db97fc46128603f29964b1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2020-09-22,ceba2e0428a67f52956045b63e2646e7637c62379ea41626571b5ade3970caea,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
curr_seq = np.asarray(test_seq[-1:]).astype(np.float64)
for i in range(-8,0):
    up_pred = model.predict(curr_seq)
    upcoming_prediction.iloc[i] = up_pred
    curr_seq = np.append(curr_seq[0][1:],up_pred,axis=0)
    curr_seq = curr_seq.reshape(test_seq[-1:].shape)

In [22]:
upcoming_prediction

,663713001,351484002,750424014,750424014,870304002,541518023,578020002,723529001,727808001,727808007,858883002,851400006
2018-09-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


__Number of articles present in this dataset__

In [24]:
max_length = len(np.unique(train['article_id']))

__Get the sequences of articles together by customer__

In [5]:
train['article_id_cart'] = train['article_id'].astype(str)
train['article_id_cart'] = train.groupby(['customer_id'])['article_id_cart'].transform(lambda x: ','.join(x))

In [6]:
train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,article_id_cart
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,"663713001,541518023,663713001,578020002,723529..."
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,"663713001,541518023,663713001,578020002,723529..."
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,"505221004,685687003,685687004,685687001,505221..."
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,"505221004,685687003,685687004,685687001,505221..."
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,"505221004,685687003,685687004,685687001,505221..."


__Group by the date and customer__

In [7]:
train = train.groupby(['t_dat','customer_id']).head()

__We no longer need these columns__

In [8]:
train.drop(['price','sales_channel_id'],axis=1,inplace=True)

In [9]:
# train = pd.DataFrame(train.drop_duplicates())

In [10]:
train.head()

,t_dat,customer_id,article_id,article_id_cart
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,"663713001,541518023,663713001,578020002,723529..."
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,"663713001,541518023,663713001,578020002,723529..."
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,"505221004,685687003,685687004,685687001,505221..."
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,"505221004,685687003,685687004,685687001,505221..."
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,"505221004,685687003,685687004,685687001,505221..."


__date_master returns the amount of article brought by that customer__

In [11]:
def date_master(article_length):
    length = len(article_length.split(","))
    return length

__date_multipler returns the day that each article was brought by that customer__

In [12]:
def date_multiplier(dt,length):
    date = dt
    temp = []
    for i in range(int(length)):
        temp.append(date)
    return ",".join(temp)

In [13]:
train['length'] = train.apply(lambda x : date_master(x['article_id_cart']),axis=1)

In [14]:
train['t_dat'] = pd.to_datetime(train['t_dat'])

In [15]:
train = train.sort_values(['customer_id','t_dat'])

In [16]:
train['nb_days'] = train.groupby('customer_id')['t_dat'].apply(lambda x: x-x.iat[0]).dt.days

In [17]:
train['nb_days'] = train.apply(lambda x : date_multiplier(str(x['nb_days']),x['length']),axis=1)

In [18]:
train.head()

,t_dat,customer_id,article_id,article_id_cart,length,nb_days
4212358,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
4212359,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
4212360,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,627759010,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
9663224,2019-05-02,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,697138006,"625548001,176209023,627759010,697138006,568601...",21,"126,126,126,126,126,126,126,126,126,126,126,12..."
10754876,2019-05-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,"625548001,176209023,627759010,697138006,568601...",21,"149,149,149,149,149,149,149,149,149,149,149,14..."


In [19]:
# train['t_dat'] = train['t_dat'].astype(str)
# train['t_dat'] = train.groupby(['customer_id'])['t_dat'].transform(lambda x: ' '.join(x))

In [20]:
# train.groupby(['customer_id']).head()

In [21]:
# train.drop(['t_dat','length'],axis=1,inplace=True)

In [22]:
# train.head()

## Feature Transformation

__Tokenization & Encoding of item_id__

In [23]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12,split=',')
tokenizer.fit_on_texts(train['article_id_cart'])
lst = tokenizer.sequences_to_texts([[1,2,3,3,4,5,6,7,8,9,10,11,12]])[0].split()
top12 = '0' + ' 0'.join(lst)
top12

'0706016001 0706016002 0610776002 0610776002 0372860001 0759871002 0610776001 0562245001 0399223001 0448509014 0372860002 0706016003'

In [24]:
# import numpy as np
# bins = np.linspace(1,100,100)
# def returning_binning(arr):
#     temp = []
#     arr = arr.split(',')
#     for i in arr:
#         temp.append(str(np.digitize(i,bins)))
#     return ",".join(temp)

In [25]:
# train['nb_days'] = train.nb_days.apply(lambda x: returning_binning(x))

__Converting Hex to Integers__

In [26]:
def hex_to_signed(source):
    """Convert a string hex value to a signed hexidecimal value.

    This assumes that source is the proper length, and the sign bit
    is the first bit in the first byte of the correct length.

    hex_to_signed("F") should return -1.
    hex_to_signed("0F") should return 15.
    """
    if not isinstance(source, str):
        raise ValueError("string type required")
    if 0 == len(source):
        raise valueError("string is empty")
    sign_bit_mask = 1 << (len(source)*4-1)
    other_bits_mask = sign_bit_mask - 1
    value = int(source, 16)
    return -(value & sign_bit_mask) | (value & other_bits_mask)

__Selecting and formating data types in dataframe__

In [27]:
train['customer_id'] = train['customer_id'].apply(lambda x : hex_to_signed(x))
#train['article_id'] = train.article_id.astype('int32')
train['t_dat'] = pd.to_datetime(train['t_dat'])
# train = train[['t_dat','customer_id','article_id']]
train.head()

,t_dat,customer_id,article_id,article_id_cart,length,nb_days
4212358,2018-12-27,9475862015264971221248214828904923560602285253...,625548001,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
4212359,2018-12-27,9475862015264971221248214828904923560602285253...,176209023,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
4212360,2018-12-27,9475862015264971221248214828904923560602285253...,627759010,"625548001,176209023,627759010,697138006,568601...",21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
9663224,2019-05-02,9475862015264971221248214828904923560602285253...,697138006,"625548001,176209023,627759010,697138006,568601...",21,"126,126,126,126,126,126,126,126,126,126,126,12..."
10754876,2019-05-25,9475862015264971221248214828904923560602285253...,568601006,"625548001,176209023,627759010,697138006,568601...",21,"149,149,149,149,149,149,149,149,149,149,149,14..."


__Saving this for later__

In [28]:
# another_train = train

__Customer's Last Week Purchases__

In [29]:
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)
train.head()

Train shape: (4112456, 8)


,t_dat,customer_id,article_id,article_id_cart,length,nb_days,max_dat,diff_dat
20,2020-09-05,9475862015264971221248214828904923560602285253...,568601043,"625548001,176209023,627759010,697138006,568601...",21,"618,618,618,618,618,618,618,618,618,618,618,61...",2020-09-05,0
84,2020-07-08,4571059672841539163133048279594898115151983489...,826211002,"583558001,639677008,640244003,521269001,666448...",86,"656,656,656,656,656,656,656,656,656,656,656,65...",2020-07-08,0
99,2020-09-15,6116990065284462810764946559699987067290722768...,794321007,"663713001,541518023,663713001,578020002,723529...",18,"726,726,726,726,726,726,726,726,726,726,726,72...",2020-09-15,0
100,2019-06-09,6393224807380828159194621156853244090628533807...,742079001,"742079001,732413001",2,"0,0",2019-06-09,0
101,2019-06-09,6393224807380828159194621156853244090628533807...,732413001,"742079001,732413001",2,"0,0",2019-06-09,0


__Recommendation Most Often Prev Purchased Items__

In [30]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,article_id_cart,length,nb_days,max_dat,diff_dat,ct
3197109,2020-09-20,-258360524981704219549932500135170439790865081...,918443002,"624634003,624634003,624634003,624634003,456163...",520,"727,727,727,727,727,727,727,727,727,727,727,72...",2020-09-20,0,15
2689872,2020-09-12,-400868796214002129303944681194737455680515276...,717464001,"816597007,816597007,858278001,866457001,866457...",52,"167,167,167,167,167,167,167,167,167,167,167,16...",2020-09-12,0,15
3637798,2020-07-07,-133918466287491330042889975257747588974461355...,820960001,"542695005,542695005,625483004,503862003,703843...",201,"651,651,651,651,651,651,651,651,651,651,651,65...",2020-07-07,0,15
6830,2019-12-10,1931192039408998770033012875146079690700007376...,727880001,"727880001,727880001,727880001,727880001,727880...",78,"3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...",2019-12-10,0,15
2639172,2019-01-05,-415205836862359829324682339179201619776164323...,628327009,"628327009,628327009,628327009,628327009,628327...",45,"6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,...",2019-01-05,0,15


In [31]:
df = train[['customer_id','article_id']]

__Recommend Items Purchased Together__

# USE PANDAS TO MAP COLUMN WITH DICTIONARY

In [32]:
pairs = np.load('pairs.npy',allow_pickle=True).item()
train['article_id2'] = train.article_id.map(pairs)
train.head()

,t_dat,customer_id,article_id,article_id_cart,length,nb_days,max_dat,diff_dat,ct,article_id2
3197109,2020-09-20,-258360524981704219549932500135170439790865081...,918443002,"624634003,624634003,624634003,624634003,456163...",520,"727,727,727,727,727,727,727,727,727,727,727,72...",2020-09-20,0,15,664074001.0
2689872,2020-09-12,-400868796214002129303944681194737455680515276...,717464001,"816597007,816597007,858278001,866457001,866457...",52,"167,167,167,167,167,167,167,167,167,167,167,16...",2020-09-12,0,15,717464002.0
3637798,2020-07-07,-133918466287491330042889975257747588974461355...,820960001,"542695005,542695005,625483004,503862003,703843...",201,"651,651,651,651,651,651,651,651,651,651,651,65...",2020-07-07,0,15,759871002.0
6830,2019-12-10,1931192039408998770033012875146079690700007376...,727880001,"727880001,727880001,727880001,727880001,727880...",78,"3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...",2019-12-10,0,15,727880003.0
2639172,2019-01-05,-415205836862359829324682339179201619776164323...,628327009,"628327009,628327009,628327009,628327009,628327...",45,"6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,...",2019-01-05,0,15,628327005.0


# RECOMMENDATION OF PAIRED ITEMS

In [33]:
train2 = train[['customer_id','article_id2']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)
train2.head()

,customer_id,article_id
3197109,-258360524981704219549932500135170439790865081...,664074001.0
2689872,-400868796214002129303944681194737455680515276...,717464002.0
3637798,-133918466287491330042889975257747588974461355...,759871002.0
6830,1931192039408998770033012875146079690700007376...,727880003.0
2639172,-415205836862359829324682339179201619776164323...,628327005.0


# CONCATENATE PAIRED ITEM RECOMMENDATION AFTER PREVIOUS PURCHASED RECOMMENDATIONS

In [34]:
# train = train[['customer_id','article_id']]
train = pd.concat([train,train2],axis=0,ignore_index=True)
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])
train.head()

,t_dat,customer_id,article_id,article_id_cart,length,nb_days,max_dat,diff_dat,ct,article_id2
0,2020-09-20,-258360524981704219549932500135170439790865081...,918443002,"624634003,624634003,624634003,624634003,456163...",520.0,"727,727,727,727,727,727,727,727,727,727,727,72...",2020-09-20,0.0,15.0,664074001.0
1,2020-09-12,-400868796214002129303944681194737455680515276...,717464001,"816597007,816597007,858278001,866457001,866457...",52.0,"167,167,167,167,167,167,167,167,167,167,167,16...",2020-09-12,0.0,15.0,717464002.0
2,2020-07-07,-133918466287491330042889975257747588974461355...,820960001,"542695005,542695005,625483004,503862003,703843...",201.0,"651,651,651,651,651,651,651,651,651,651,651,65...",2020-07-07,0.0,15.0,759871002.0
3,2019-12-10,1931192039408998770033012875146079690700007376...,727880001,"727880001,727880001,727880001,727880001,727880...",78.0,"3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...",2019-12-10,0.0,15.0,727880003.0
4,2019-01-05,-415205836862359829324682339179201619776164323...,628327009,"628327009,628327009,628327009,628327009,628327...",45.0,"6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,...",2019-01-05,0.0,15.0,628327005.0


# CONVERT RECOMMENDATIONS INTO SINGLE STRING

In [35]:
train.article_id = ' 0' + train.article_id.astype('str')
preds = pd.DataFrame( train.groupby('customer_id').article_id.sum().reset_index() )
preds.columns = ['customer_id','prediction']
preds.head()

,customer_id,prediction
0,-578959872699432204840272251318678134742912003...,0918522001 0915526001
1,-578958955106318985703340409806304793781964660...,0918443004 0158340001
2,-578958851022090388530167605110184806675590806...,0734460001 0898983001 0469039019 0706016001 0...
3,-578958530827022703791934662315899566725724297...,0429313023 0429313025 0679284001 0429313008 0...
4,-578954918733087932666888089058340064531866922...,0696527001 0266875001 0554772041 0617534016 0...


In [36]:
# train.t_dat = pd.to_datetime(train.t_dat)
# train = train.loc[train.t_dat >= pd.to_datetime('2020-09-16')]
# top12 = ' 0' + ' 0'.join(train.article_id.value_counts().index.astype('str')[:12])
print("Last week's top 12 popular items:")
print( top12 )

Last week's top 12 popular items:
0706016001 0706016002 0610776002 0610776002 0372860001 0759871002 0610776001 0562245001 0399223001 0448509014 0372860002 0706016003


In [37]:
sub = pd.read_csv("C:/Users/Michael/Github/sample_submission.csv")
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].apply(lambda x : hex_to_signed(x))
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1),on='customer_id_2', how='left').fillna('')
del sub['customer_id_2']
sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]
#sub.to_csv(f'submission.csv',index=False)
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 05686010060706016001 0706016002 061...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 07060160010706016001 0706016002 061...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 08050000010706016001 0706016002 061...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001 0730683001 0372860001070...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015 08...


In [38]:
sub.to_csv(f'submission_05_07_2022.csv',index=False)